In [1]:
import cvxpy as cp
import numpy as np
import osbdo as ob

# Hello world example
$$\begin{array}{ll}
\mbox{minimize } &\left((1/2)x_1^TP_1x_1+q_1^Tx_1+r_1\right)+\left(c_2^Tx_2+d_2\right)+\left((1/2)x_3^TP_3x_3+q_3^Tx_3+r_3\right)\\
\mbox{subject to }&l_1\leq x_1\leq u_1\\
&l_2\leq x_2\leq u_2\\
&l_3\leq x_3\leq u_3\\
&(x_{1})_1+2(x_2)_1=(x_3)_1
\end{array}$$

# Describe each $f_i$ in a class `Agent_i(osbdo.Agent)`

In [2]:
class AffineAgent(ob.Agent):
    """
       objective: c^T x + d
    """    
    def _construct_params(self):
        self.c = self.params['c']
        self.d = self.params['d']
        
    def query(self, *, v, solver):
        return ob.Point(x=v, q=self.c, f=(self.c.T@v+self.d).sum())
    
    def get_init_minorant(self):
        var = cp.Variable(self.dim)
        prob = cp.Problem(cp.Minimize(self.c.T@var+self.d),[var<=self.upb, var>=self.lwb])
        prob.solve()
        self.lwb_const = prob.value
        return cp.Constant(prob.value)

            
class QuadraticAgent(ob.Agent):
    """
       objective:(1/2) x^T P x + q^T x + r
    """
    def _construct_params(self):
        self.P = self.params['P']
        self.q = self.params['q']
        self.r = self.params['r']

    def query(self, *, v, solver):
        return ob.Point(x=v, q=self.P@v+self.q, f=(0.5*v.T@self.P@v+self.q.T@v+self.r).sum())
    
    def get_init_minorant(self):
        var = cp.Variable(self.dim)
        prob = cp.Problem(cp.Minimize(0.5*cp.quad_form(var,self.P)+self.q.T@var+self.r),\
                                                                    [var<=self.upb, var>=self.lwb])
        prob.solve()
        self.lwb_const = prob.value
        return cp.Constant(prob.value)

### Create 3 agents

The public variables have size $x_1  \in {\mathbf R}^5,~ x_2 \in {\mathbf R}^3,~ x_3 \in {\mathbf R}$.

In [3]:
# define agent 1
P = np.random.rand(5,5)
P1 = P.T@P
params1 = {'dimension':5, 'lower_bound':np.array([-2,-3,2,-6,-10]), 'upper_bound':np.array([1,3,6,9,-5]),
          'P': P1, 'q':np.random.rand(5), 'r':np.random.rand()}
tetiana = QuadraticAgent(params1)

# define agent 2
params2 = {'dimension':3, 'lower_bound':np.array([-2,6,-5]), 'upper_bound':np.array([5,10,-1]), 
          'c':np.random.rand(3), 'd':np.random.rand()}
fangzhao = AffineAgent(params2)

# define agent 3
P = np.random.rand(1,1)
P3 = P.T@P
params3 = {'dimension':1, 'lower_bound': np.array([-2]), 'upper_bound': np.array([17]),
          'P': P3, 'q': np.random.rand(1), 'r': np.random.rand()}
stephen = QuadraticAgent(params3)

In [4]:
agents = [tetiana, fangzhao, stephen]

# Coupling

In [5]:
domain = [tetiana.x[0] + 2 * fangzhao.x[0] == stephen.x]
g = ob.Coupling(agents=agents, \
                function = cp.Constant(0), domain = domain)

# Problem

In [6]:
prob = ob.Problem(agents = agents, g = g)

# Solve problem

In [7]:
x_agents, x_global = prob.solve()

k=0, rel_gap=2.9838230923369116, L=5.112779881297483, U=20.368410557148486, L0=5.112779881297483
k=1, rel_gap=2.8483646707700916, L=5.292744399161265, U=20.368410557148486, L0=5.292744399161265
k=2, rel_gap=2.8043138969201684, L=5.29274439988923, U=20.135261073344996, L0=5.29274439988923
k=3, rel_gap=1.8122031220997188, L=5.29274439988923, U=14.884272325844295, L0=5.292744399040429


k=4, rel_gap=1.280956738770169, L=5.380411429785339, U=12.27248570812491, L0=5.380411429785339
k=5, rel_gap=0.8001075647459097, L=5.562411684500406, U=10.012939351500219, L0=5.562411684500406
k=6, rel_gap=0.5468747432495842, L=5.606328930237508, U=8.672288624533861, L0=5.606328930237508
k=7, rel_gap=0.31812884084839677, L=5.755223365865451, U=7.586125904071835, L0=5.755223365865451
k=8, rel_gap=0.18605746247102845, L=5.793787959725707, U=6.871765445607469, L0=5.793787959725707
k=9, rel_gap=0.10426960027407321, L=5.830301497374655, U=6.4382247039832405, L0=5.830301497374655
k=10, rel_gap=0.09848540457630436, L=5.861001591064855, U=6.4382247039832405, L0=5.861001591064855
k=11, rel_gap=0.061642389922367466, L=5.868201391306227, U=6.229931349612105, L0=5.868201391306227
k=12, rel_gap=0.03562680561347408, L=5.874905469679945, U=6.084209584845768, L0=5.874905469679945
k=13, rel_gap=0.02338794426376355, L=5.874905469679945, U=6.012307431359699, L0=5.87490546899669
k=14, rel_gap=0.01381407132

# Centralized solution

In [8]:
x1 = cp.Variable(5)
x2 = cp.Variable(3)
x3 = cp.Variable(1)

f1 = 0.5 * cp.quad_form(x1,params1['P']) + params1['q'].T@x1 + params1['r']
f2 = params2['c'].T@x2 + params2['d']
f3 = 0.5 * cp.quad_form(x3,params3['P']) + params3['q'].T@x3 + params3['r']

constr1 = [x1 >= params1['lower_bound'], x1 <= params1['upper_bound']]
constr2 = [x2 >= params2['lower_bound'], x2 <= params2['upper_bound']]
constr3 = [x3 >= params3['lower_bound'], x3 <= params3['upper_bound']]
constr4 = [x1[0] + 2*x2[0] == x3]

cvx_prob = cp.Problem(cp.Minimize(f1+f2+f3), constr1+constr2+constr3+constr4)
cvx_prob.solve()

5.883623195911768

# Comparison

In [9]:
x_agents

[array([-1.99202073,  2.8980713 ,  2.1523286 ,  0.74561374, -5.        ]),
 array([-3.98963339e-03,  6.00000000e+00, -4.89726857e+00]),
 array([-2.])]

In [10]:
x1.value, x2.value, x3.value

(array([-2.        ,  2.91899752,  2.19170551,  0.72887879, -5.        ]),
 array([-3.09666421e-17,  6.00000000e+00, -5.00000000e+00]),
 array([-2.]))

# $L^k, \quad h^\star, \quad U^k$

In [11]:
prob.lower_bnd[-1], cvx_prob.value, prob.upper_bnd[-1]

(5.881385480405349, 5.883623195911768, 5.923640680823684)